In [14]:
!git clone https://github.com/jackrubin1/econ8310-assignment1.git
%cd econ8310-assignment1

Cloning into 'econ8310-assignment1'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 95 (delta 36), reused 27 (delta 27), pack-reused 50 (from 1)
Receiving objects: 100% (95/95), 100.14 KiB | 3.58 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1/econ8310-assignment1


In [27]:
import pandas as pd
import numpy as np
import os
from pygam import LinearGAM, s, f

train_path = "assignment_data_train.csv"
train_url = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_train.csv"

if os.path.exists(train_path):
    train_data = pd.read_csv(train_path)
else:
    train_data = pd.read_csv(train_url)

train_data['Timestamp'] = pd.to_datetime(train_data['Timestamp'])
train_data = train_data.sort_values('Timestamp')

x_train = train_data[['year', 'month', 'day', 'hour']].values
y_train = train_data['trips'].values

model = LinearGAM(s(0) + f(1) + f(2) + s(3))

modelFit = model.gridsearch(x_train, y_train)

test_path = "assignment_data_test.csv"
test_url = "https://github.com/dustywhite7/econ8310-assignment1/raw/main/assignment_data_test.csv"

if os.path.exists(test_path):
    test_data = pd.read_csv(test_path)
else:
    test_data = pd.read_csv(test_url)

test_data['Timestamp'] = pd.to_datetime(test_data['Timestamp'])
test_data = test_data.sort_values('Timestamp')
x_test = test_data[['year', 'month', 'day', 'hour']].values

pred = modelFit.predict(x_test)

pred = np.maximum(pred, 0)

100% (11 of 11) |########################| Elapsed Time: 0:00:02 Time:  0:00:02
